In [53]:
import requests

api_key="D43F253587FB437283F89A4EDF2CD6DC"
octoprint_server="http://imi-octopi01.imi.kit.edu/api/"

headers = {'X-Api-Key': api_key, 'Content-Type': 'application/json'}

In [54]:
JOB = "job"
BED = "printer/bed"
PRINTER = "printer"
PRINTHEAD = "printer/printhead"

In [55]:
response = requests.get(octoprint_server+PRINTER, headers=headers)
response.json()

{'sd': {'ready': False},
 'state': {'error': '',
  'flags': {'cancelling': False,
   'closedOrError': False,
   'error': False,
   'finishing': False,
   'operational': True,
   'paused': False,
   'pausing': False,
   'printing': False,
   'ready': True,
   'resuming': False,
   'sdReady': False},
  'text': 'Operational'},
 'temperature': {'A': {'actual': 42.75, 'offset': 0, 'target': 0.0},
  'W': {'actual': 0.0, 'offset': 0, 'target': None},
  'X': {'actual': 33.77, 'offset': 0, 'target': 36.0},
  'bed': {'actual': 73.06, 'offset': 0, 'target': 0.0},
  'tool0': {'actual': 127.9, 'offset': 0, 'target': 0.0}}}

In [9]:
print(response.status_code)  # To print the status code of the response
print(response.json())       # To print the JSON content of the response, if any

200
{'job': {'averagePrintTime': None, 'estimatedPrintTime': None, 'filament': None, 'file': {'date': None, 'display': None, 'name': None, 'origin': None, 'path': None, 'size': None}, 'lastPrintTime': None, 'user': None}, 'progress': {'completion': None, 'filepos': None, 'printTime': None, 'printTimeLeft': None, 'printTimeLeftOrigin': None}, 'state': 'Operational'}


In [20]:
response = requests.get(octoprint_server+BED, headers=headers)
response.json()

{'bed': {'actual': 23.93, 'offset': 0, 'target': 0.0}}

In [50]:
response = requests.get(octoprint_server+BED, headers=headers)
response.json()

{'error': 'The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.'}

In [43]:
def home_printer(axes = ["x", "y","z"]):
    """Set the printer to the home position"""
    payload = {
  "command": "home",
  "axes": axes
}
    response = requests.post(octoprint_server+PRINTHEAD, headers=headers, json=payload)
        # Print status code and raw response content
    print("Status Code:", response.status_code)
    print("Raw Response Content:", response.text)
    
    # Try to parse JSON response if possible
    try:
        response_json = response.json()
        print(response_json)
    except ValueError:
        print("Response content is not valid JSON")
    return response

def jog_printer(x=10, y=10, z=0.05):
    """Move the printer along specified axes"""
    payload = {
        "command": "jog",
        "x": x,
        "y": y,
        "z": z
    }
    response = requests.post(octoprint_server + PRINTHEAD, headers=headers, json=payload)
    
    # Print status code and raw response content
    print("Status Code:", response.status_code)
    print("Raw Response Content:", response.text)
    
    # Try to parse JSON response if possible
    try:
        response_json = response.json()
        print(response_json)
    except ValueError:
        print("Response content is not valid JSON")
    
    return response

In [46]:
response = jog_printer(y=100)

Status Code: 204
Raw Response Content: 
Response content is not valid JSON


In [44]:
response = home_printer()

Status Code: 204
Raw Response Content: 
Response content is not valid JSON


In [57]:
import requests

PRINTHEAD_COMMAND_ENDPOINT = "/printer/command"

def send_gcode(command):
    payload = {
        "command": command
    }
    response = requests.post(octoprint_server + PRINTHEAD_COMMAND_ENDPOINT, headers=headers, json=payload)
    
    # Verify the response
    if response.status_code == 204:
        print("Command successfully sent")
    else:
        print("Error while sending command")
        print("Status code:", response.status_code)
        print("Response contents:", response.text)
    
    return response

# Use example
send_gcode("G28")  # "homing" command
send_gcode("G1 X10 Y10 Z0.3 F3000")  # Moving command


Command successfully sent
Command successfully sent


<Response [204]>

In [59]:
send_gcode("G1 Z10 F3000")

Command successfully sent


<Response [204]>

In [60]:
square_commands = [
    "G28",           # Home all axes
    "G1 Z10 F3000",  # Move the print head up to avoid any obstacles
    "G1 X0 Y0 F3000",# Move to the start position
    "G1 X10 Y0 F3000", # Move to (10, 0)
    "G1 X10 Y10 F3000", # Move to (10, 10)
    "G1 X0 Y10 F3000",  # Move to (0, 10)
    "G1 X0 Y0 F3000",   # Move to (0, 0)
    "G1 Z0.3 F3000"     # Move the print head back down to the print level
]

In [62]:
for command in square_commands:
    send_gcode(command)

Command successfully sent
Command successfully sent
Command successfully sent
Command successfully sent
Command successfully sent
Command successfully sent
Command successfully sent
Command successfully sent


In [64]:
def read_gcode(fichier):
    try:
        with open(fichier, 'r') as f:
            lignes = f.readlines()
        
        premieres_parties = []
        for ligne in lignes:
            ligne = ligne.strip()
            parties = ligne.split(';')
            if len(parties) > 0 and len(parties[0])!=0:
                premieres_parties.append(parties[0])
        
        return premieres_parties
    
    except FileNotFoundError:
        print(f"The file {fichier} hasn't been found.")
        return []
    except Exception as e:
        print(f"An error happened: {e}")
        return []

# Exemple d'utilisation
file = 'TEST_Forme-Boite_0.4n_0.2mm_PETG_MK4IS_1m.gcode'
gcode_list = read_gcode(file)
print(gcode_list)


['M73 P0 R1', 'M73 Q0 S1', 'M201 X4000 Y4000 Z200 E2500 ', 'M203 X300 Y300 Z40 E100 ', 'M204 P4000 R1200 T4000 ', 'M205 X8.00 Y8.00 Z2.00 E10.00 ', 'M205 S0 T0 ', 'M486 S0', 'M486 AForme-Boîte', 'M486 S-1', 'M17 ', 'M862.1 P0.4 ', 'M862.3 P "MK4" ', 'M862.5 P2 ', 'M862.6 P"Input shaper" ', 'M115 U6.0.1+14848', 'M555 X122.463 Y97.0863 W32 H11.8275', 'G90 ', 'M83 ', 'M140 S85 ', 'M104 T0 S175 ', 'M109 T0 R175 ', 'M84 E ', 'G28 ', 'G1 X42 Y-4 Z5 F4800', 'M302 S160 ', 'G1 E-2 F2400 ', 'M73 P1 R1', 'M73 Q1 S1', 'M84 E ', 'G29 P9 X10 Y-4 W32 H4', 'G0 Z40 F10000', 'M190 S85 ', 'M107', 'M84 E ', 'G29 P1 ', 'G29 P1 X0 Y0 W50 H20 C ', 'G29 P3.2 ', 'G29 P3.13 ', 'G29 A ', 'M104 S240', 'G0 X0 Y-4 Z15 F4800 ', 'M73 P2 R1', 'M73 Q2 S1', 'M109 S240', 'G92 E0', 'M569 S0 E ', 'G92 E0 ', 'G1 E2 F2400 ', 'M73 P3 R1', 'M73 Q3 S1', 'G0 E7 X15 Z0.2 F500 ', 'G0 X25 E4 F500 ', 'M73 P7 R1', 'M73 Q7 S1', 'G0 X35 E4 F650 ', 'M73 P8 R1', 'M73 Q8 S1', 'G0 X45 E4 F800 ', 'M73 P9 R1', 'M73 Q9 S1', 'G0 X48 Z0.05 F800

In [65]:
def print_3D(object_3D):
    for line in object_3D:
        send_gcode(line)

In [66]:
print_3D(gcode_list)

Command successfully sent
Command successfully sent
Command successfully sent
Command successfully sent
Command successfully sent
Command successfully sent
Command successfully sent
Command successfully sent
Command successfully sent
Command successfully sent
Command successfully sent
Command successfully sent
Command successfully sent
Command successfully sent
Command successfully sent
Command successfully sent
Command successfully sent
Command successfully sent
Command successfully sent
Command successfully sent
Command successfully sent
Command successfully sent
Command successfully sent
Command successfully sent
Command successfully sent
Command successfully sent
Command successfully sent
Command successfully sent
Command successfully sent
Command successfully sent
Command successfully sent
Command successfully sent
Command successfully sent
Command successfully sent
Command successfully sent
Command successfully sent
Command successfully sent
Command successfully sent
Command succ